In [1]:
import os
import pandas as pd
import numpy as np
import timeit
import pickle

# Functions

In [2]:
# def get_junction_coordinates(df, coordinates_col, sep=':', shift_to_1based=True):
#     df['strand'] = None
#     df['junction_coordinate'] = None

#     for idx, row in df.iterrows():
#         kmer_coordinates = [int(x) for x in row[coordinates_col].split(sep) if x !='None']

#         if kmer_coordinates[1] < kmer_coordinates[2]: # order strand +

#             df.loc[idx, 'strand'] = '+'
#             if len(kmer_coordinates) == 4:  # 2 exons
#                 df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in kmer_coordinates[1:3]])
#             elif len(kmer_coordinates) == 6:
#                 df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in kmer_coordinates[1:5]])
#         else: # order strand +
#             df.loc[idx, 'strand'] = '-'
#             if len(kmer_coordinates) == 4:  # 2 exons
#                 df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in [kmer_coordinates[3],
#                                                                                 kmer_coordinates[0]]])
#             elif len(kmer_coordinates) == 6:
#                 df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in [kmer_coordinates[3],
#                                                                                 kmer_coordinates[0],
#                                                                                 kmer_coordinates[2],
#                                                                                 kmer_coordinates[5]
#                                                                                ]])
#     return df




In [3]:
def explode_immunopepper_coord2(mx, coord_col = 'coord', sep=':'):
    
    coord_mx = mx[coord_col].str.split(sep, expand=True) #7 min
    
    dummy_fill = -9999
    coord_mx = coord_mx.astype(float)
    coord_mx.fillna(dummy_fill, inplace=True)
    coord_mx = coord_mx.astype(int)

    coord_mx['strand'] = None
    coord_mx.loc[coord_mx[1] < coord_mx[2], 'strand'] = '+'
    coord_mx.loc[coord_mx[1] > coord_mx[2], 'strand'] = '-'

    coord_mx['junction_coordinate1'] = None
    coord_mx['junction_coordinate2'] = None

    if 4 not in coord_mx.columns:
        coord_mx[4] = dummy_fill
        coord_mx[5] = dummy_fill
    
    coord_mx = coord_mx.astype(str) # 7 min
    
    coord_mx['+first'] = coord_mx[1] + ':' + coord_mx[2]
    coord_mx['+secon'] = coord_mx[3] + ':' + coord_mx[4]
    coord_mx['-first'] = coord_mx[3] + ':' + coord_mx[0]
    coord_mx['-secon'] = coord_mx[5] + ':' + coord_mx[2]
    
        
    coord_mx[4] = ['None' if str(dummy_fill) in jx2 else jx2 for jx2 in coord_mx[4]]
    coord_mx[5] = ['None' if str(dummy_fill) in jx2 else jx2 for jx2 in coord_mx[5]]

    coord_mx.loc[(coord_mx['strand'] == '+'), 'junction_coordinate1'] = coord_mx['+first'] 
    coord_mx.loc[(coord_mx['strand'] == '-'), 'junction_coordinate1'] = coord_mx['-first'] 
    coord_mx.loc[(coord_mx['strand'] == '+') & (coord_mx[4] != 'None') , 'junction_coordinate2'] = coord_mx['+secon']
    coord_mx.loc[(coord_mx['strand'] == '-') & (coord_mx[4] != 'None') , 'junction_coordinate2'] = coord_mx['-secon']
    
    return coord_mx

In [4]:
def explode_immunopepper_coord(mx, coord_col = 'coord', sep=':'):
    coord_mx = mx[coord_col].str.split(sep, expand=True) #7 min

    coord_mx[1] = coord_mx[1].astype('int')
    coord_mx[2] = coord_mx[2].astype('int')

    coord_mx['strand'] = None
    coord_mx.loc[coord_mx[1] < coord_mx[2], 'strand'] = '+'
    coord_mx.loc[coord_mx[1] > coord_mx[2], 'strand'] = '-'

    coord_mx['junction_coordinate1'] = None
    coord_mx['junction_coordinate2'] = None

    
    coord_mx = coord_mx.astype(str) # 7 min
    
    coord_mx['+first'] = coord_mx[1] + ':' + coord_mx[2]
    coord_mx['+secon'] = coord_mx[3] + ':' + coord_mx[4]
    coord_mx['-first'] = coord_mx[3] + ':' + coord_mx[0]
    coord_mx['-secon'] = coord_mx[5] + ':' + coord_mx[2]
    


    coord_mx.loc[(coord_mx['strand'] == '+'), 'junction_coordinate1'] = coord_mx['+first'] 
    coord_mx.loc[(coord_mx['strand'] == '-'), 'junction_coordinate1'] = coord_mx['-first'] 
    coord_mx.loc[(coord_mx['strand'] == '+') & (coord_mx[4] != 'None') , 'junction_coordinate2'] = coord_mx['+secon']
    coord_mx.loc[(coord_mx['strand'] == '-') & (coord_mx[4] != 'None') , 'junction_coordinate2'] = coord_mx['-secon']
    
    return coord_mx

# Main

In [26]:
# Paths INPUTS

sample_type = 'brca'

if sample_type == 'brca':
    sample_dir = 'TCGA_Breast_1102'
elif sample_type == 'ov':
    sample_dir = 'TCGA_Ovarian_374'

basedir = f'/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/{sample_dir}'
print(basedir)
intermediate_file = os.path.join(basedir, 'filtering_intermediate/complete_cancer_candidates_order_r_complete.tsv.gz')

longlist = os.path.join(basedir, 'filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/All_samples_longlist.tsv.gz')

save_ = True
dump_ = True


/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102


In [27]:
# Paths OUTPUTS
csv_res = os.path.join(basedir, 'filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/peptides_25012024.tsv.gz') 
pickle_res = os.path.join(basedir, 'filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/peptides25012024.pickle')

In [28]:
# Load
inter = pd.read_csv(intermediate_file, sep = '\t')
long_list = pd.read_csv(longlist, sep = '\t', header = None)
long_list.columns = ['kmer', 'coord']

In [29]:
display(inter.head()) 
inter.shape

display(long_list.head()) 
long_list.shape

,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,cancerCohortfilter >=2.0,cancerCohortfilter >=3.0,cancerCohortfilter >=5.0,cancerCohortfilter >=10.0,TCGAC8A12P01A11RA11507all,TCGAAOA0JM01A21RA05607all,TCGABHA18V01A11RA12D07all,TCGAA2A0D201A21RA03407all,TCGAA2A0SX01A12RA08407all,isAnnotated
0,WYITRSGIA,3290.0,1969.0,861.0,394.0,89.0,4.0,92347505:92347506:92349915:92349941:None:None,False,False,...,138,82,26,5,0.000000,0.000000,0.000000,4.102634,0.0,NaN
1,WYITRSGIA,572.0,545.0,243.0,130.0,27.0,2.0,92347505:92347506:92349915:92349941:None:None,False,False,...,138,82,26,5,0.000000,0.000000,0.000000,4.102634,0.0,NaN
2,ISSQSRVEK,278.0,261.0,82.0,29.0,7.0,0.0,92379851:92379859:92493866:92493885:None:None,False,False,...,10,2,0,0,0.000000,0.000000,2.474321,0.000000,0.0,NaN
3,RSGDEEKYP,7350.0,4622.0,2653.0,1745.0,1126.0,734.0,92600493:92600508:92611313:92611325:None:None,True,True,...,641,520,348,170,2.922641,2.102386,1.237161,0.000000,0.0,1.0
4,HLKMKMFQI,146.0,136.0,42.0,20.0,3.0,1.0,92379850:92379859:92496416:92496434:None:None,False,False,...,16,2,0,0,2.922641,0.000000,0.000000,0.000000,0.0,NaN


,kmer,coord
0,AAASFSMDG,62492703:62492724:62477828:62477834:None:None
1,AACFLVHPS,62270267:62270271:62270854:62270877:None:None
2,AADDISAQL,224929828:224929835:225085709:225085729:None:None
3,AAETLQDDF,129536369:129536377:129540389:129540408:None:None
4,AAGDSPIPG,177436312:177436319:177434461:177434481:None:None


(5579, 2)

In [30]:
# Merge on metadata 
batch_interest = inter.merge(long_list, on = ['kmer', 'coord'], how = 'right')

print(batch_interest.shape)

print(len(batch_interest['batch'].unique()))

(6080, 24)
662


In [31]:
# Add Coordinate
coord_mx = explode_immunopepper_coord(batch_interest)

batch_interest = pd.concat([batch_interest, 
                            coord_mx[['strand', 'junction_coordinate1', 'junction_coordinate2']]], axis = 1)

In [32]:
batch_interest.shape

(6080, 27)

In [33]:
batch_interest.head()

,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,cancerCohortfilter >=10.0,TCGAC8A12P01A11RA11507all,TCGAAOA0JM01A21RA05607all,TCGABHA18V01A11RA12D07all,TCGAA2A0D201A21RA03407all,TCGAA2A0SX01A12RA08407all,isAnnotated,strand,junction_coordinate1,junction_coordinate2
0,AAASFSMDG,9.0,6.0,4.0,3.0,1.0,0.0,62492703:62492724:62477828:62477834:None:None,False,False,...,0,1.461321,0.0,0.0,0.000000,0.0,NaN,-,62477834:62492703,None
1,AACFLVHPS,4.0,4.0,2.0,2.0,0.0,0.0,62270267:62270271:62270854:62270877:None:None,False,True,...,6,10.229245,0.0,0.0,0.000000,0.0,NaN,+,62270271:62270854,None
2,AADDISAQL,12.0,11.0,3.0,0.0,0.0,0.0,224929828:224929835:225085709:225085729:None:None,False,False,...,0,0.000000,0.0,0.0,1.367545,0.0,NaN,+,224929835:225085709,None
3,AAETLQDDF,9.0,9.0,5.0,4.0,0.0,0.0,129536369:129536377:129540389:129540408:None:None,False,True,...,0,2.922641,0.0,0.0,0.000000,0.0,NaN,+,129536377:129540389,None
4,AAETLQDDF,9.0,9.0,5.0,4.0,0.0,0.0,129536369:129536377:129540389:129540408:None:None,False,False,...,0,2.922641,0.0,0.0,0.000000,0.0,NaN,+,129536377:129540389,None


In [34]:
# Merge peptides 
debut_time = timeit.default_timer()
tot_shape = 0
extracted_metadata = []
for idx, gene in enumerate(batch_interest['batch'].unique()):
    # Load metadata
    base_meta = os.path.join(basedir, 'cohort_mutNone', f'tmp_out_ref_batch_{gene}', 'ref_sample_peptides_meta.gz')
    meta = pd.read_csv(base_meta, sep = '\t') # Note they are peptides 'NA' confused with Nan 
    meta = meta[~meta['peptide'].isna()]
    meta = meta.set_index(np.arange(len(meta)))
    meta_coord = explode_immunopepper_coord2(meta, coord_col='modifiedExonsCoord', sep=';')
    meta = pd.concat([meta, meta_coord[['strand', 'junction_coordinate1', 'junction_coordinate2']]], axis = 1)
    
    # Merge with coordinates interest
    batch_col = ['coord', 'kmer', 'junction_coordinate1', 'junction_coordinate2']
#     selected_pep = batch_interest.loc[:, batch_col].merge(meta, 
#                                         on = ['junction_coordinate1', 'junction_coordinate2'], 
#                                         how = 'inner')

    selected_pep0 = batch_interest.loc[:, batch_col].merge(meta, 
                                                         on = ['junction_coordinate1', 
                                                               'junction_coordinate2'], 
                                                         how = 'inner')
    selected_pep1 = batch_interest.loc[:, batch_col].merge(meta, #correct issue with second coord only   
                                                          right_on = [ 'junction_coordinate2'], 
                                                          left_on = ['junction_coordinate1'], 
                                                         how = 'inner')
    selected_pep2 = batch_interest.loc[:, batch_col].merge(meta.loc[meta['kmerType'] == '3-exons'], #correct issue with first coordinate from 3-exons
                                                          on = ['junction_coordinate1'], 
                                                         how = 'inner')
    selected_pep = pd.concat([selected_pep2, selected_pep1, selected_pep0]).\
                drop_duplicates().reset_index()
    # Restrict merge on kmer presence
    selected_pep['keep'] = [selected_pep.loc[ix, 'kmer'] in pep 
                            for ix, pep in enumerate(selected_pep['peptide'])]
    selected_pep = selected_pep.loc[selected_pep['keep'] == True]
    
    # Prepare save
    extracted_metadata.append(selected_pep)
    tot_shape+=selected_pep.shape[0]
    
    if idx % 100 == 0:
        print(f'....{idx} gene number {gene} total peptides {tot_shape} add {selected_pep.shape[0]}')
        print
print(timeit.default_timer() - debut_time)
print(tot_shape)


....0 gene number 1841 total peptides 22 add 22
....100 gene number 34142 total peptides 5198 add 13
....200 gene number 53915 total peptides 7191 add 24
....300 gene number 28987 total peptides 8508 add 4
....400 gene number 22200 total peptides 10849 add 21
....500 gene number 55165 total peptides 12379 add 20
....600 gene number 1167 total peptides 13969 add 5
69.7245497489348
14529


In [35]:
len(extracted_metadata)

662

In [36]:
if dump_: 
    print(f'Saving to {pickle_res}')
    file_ = open(pickle_res, 'wb')

    # dump information to that file
    pickle.dump(extracted_metadata, file_)

Saving to /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/peptides25012024.pickle


In [37]:
res = pd.concat(extracted_metadata, axis = 0)

In [38]:
if save_:
    print(f'Saving to {csv_res}')
    res.to_csv(csv_res, 
              sep = '\t', 
              index= None, 
              compression = 'gzip')

Saving to /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/peptides_25012024.tsv.gz


In [43]:
meta.loc[meta['id'] == 'ENSG00000167996.16:1775_1290:0:61965419:2-exons', 'peptide'].isna()

18378    True
Name: peptide, dtype: bool

# TESTS